In [1]:
using TSSM
include("time_stepper.jl");

In [2]:
function get_coeffs_composition(g::Vector{Float64})
    n = length(g)
    z = zeros(n)
    a = zeros(n+1)
    b = vcat(g, 0)
    h = 0.0
    z[div(n,2)+1] = 0.5
    for j=1:div(n,2)
        z[j] = h+g[j]/2
        z[n-j+1] = 1-z[j]
        h += g[j]
    end
    a[1] = g[1]/2
    a[n+1] = a[1]
    for j=2:div(n,2)+1
        a[j] = (g[j-1]+g[j])/2
        a[n-j+2] = a[j]
    end
    a,b,z
end

get_coeffs_composition (generic function with 1 method)

In [3]:
function gen_interpolation_matrix(x::Vector{Float64}, z::Vector{Float64})
    n = length(x)
    m = length(z)    
    L = zeros(n, m)
    for j=1:m
        for i=1:n
            L[i,j] = prod([(z[j]-x[k])/(x[i]-x[k]) for k=1:i-1])*prod([(z[j]-x[k])/(x[i]-x[k]) for k=i+1:n])
        end
    end
    L
end

gen_interpolation_matrix (generic function with 1 method)

In [4]:
function gen_midpoint!(midpoint::WfSchroedinger1D, L::Vector{Float64}, psi_back::Vector{WfSchroedinger1D}, first::Int)
    n = length(psi_back)
    set!(midpoint, 0)
    for j=1:n
        k = mod(j+first-2, n)+1
        axpy!(midpoint, psi_back[k], L[j])
    end
end

gen_midpoint! (generic function with 1 method)

In [5]:
function TSSM.propagate_B!(psi::WfSchroedinger1D, dt::Number, F::Function, midpoint::WfSchroedinger1D)
    to_real_space!(psi)
    to_real_space!(midpoint)
    u = get_data(psi, true)
    u1 = get_data(midpoint, true)
    for j=1:length(u)
        u[j] += dt*F(u1[j])
    end
end

propagate_B! (generic function with 22 methods)

In [6]:
function run(F::Function, dt::Number, N::Int, psi_back::Vector{WfSchroedinger1D}, g::Vector{Float64})
    n = length(psi_back)
    m = psi_back[1].m
    psi = wave_function(m)
    midpoint = wave_function(m)
    copy!(psi, psi_back[n])
    a,b,z = get_coeffs_composition(g)
    s = length(g)
    L = gen_interpolation_matrix(collect(-(n-1.0):0.0), z)
    first = 1
    for k = n:N
        for j = 1:s
            propagate_A!(psi, a[j]*dt)
            gen_midpoint!(midpoint, L[:,j], psi_back, first)
            propagate_B!(psi, b[j]*dt, F, midpoint)
        end    
        propagate_A!(psi, a[s+1]*dt)                
        copy!(psi_back[first], psi)
        to_real_space!(psi_back[first])
        first = mod(first, n) + 1
    end
    psi
end

run (generic function with 1 method)

In [7]:
function gen_starting_values(psi::WfSchroedinger1D, dt::Number, N::Int, g::Vector{Float64})
    a,b,z = get_coeffs_composition(g)
    s = length(g)
    psi_back = WfSchroedinger1D[wave_function(m) for j=1:N]
    copy!(psi_back[1], psi)
    for k=2:N
        for j = 1:s
            propagate_A!(psi, a[j]*dt)
            propagate_B!(psi, b[j]*dt)
        end
        propagate_A!(psi, a[s+1]*dt)
        copy!(psi_back[k], psi)
    end
    psi_back
end

gen_starting_values (generic function with 1 method)

In [8]:
nx = 2048
xmin = -16
xmax = +16
m = Schroedinger1D(nx, xmin, xmax, cubic_coupling=-1)

TSSM.Schroedinger1D{Float64}(Ptr{Void} @0x00000000026b8780)

In [9]:
# cubic_coupling=-1 has to be multiplied by -1im 
# because of the factor 1im at the lefthand side of the Schrödinger equation
F(u) = 1im*conj(u)*u^2

F (generic function with 1 method)

In [10]:
# exact solution
const a = 2
const b = 1
const c = 0
function soliton(x, t)
    h = (a^2 - b^2)/2*t - b*x
    (a./cosh(a*(b*t+x-c))).*exp(1im*h) 
end    

soliton (generic function with 1 method)

In [11]:
t0 = 0
tend = 1
psi = wave_function(m)
psi_ref = wave_function(m)
set!(psi, soliton, t0)        # initial data at t=t0
set!(psi_ref, soliton, tend)  # reference solution at t=tend

In [12]:
set!(psi, soliton, t0)
#g = [1/(2-2^(1/3)),-2^(1/3)/(2-2^(1/3)), 1/(2-2^(1/3))] # Yoshida
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
N = 512
dt = (tend-t0)/N
psi_back = gen_starting_values(psi, dt, 5, g);
psi = run(F, dt, N, psi_back, g)
distance(psi, psi_ref)

3.507350738832297e-10

In [13]:
1.0640857358040506e-7/5.56784149074978e-9

19.111279255558657

In [14]:
5.56784149074978e-9/3.507350738832297e-10

15.874778159778462